# Collaborative Filtering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Info

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks/transactions_train.csv', nrows=50000)

In [ ]:
articles = pd.read_csv('drive/MyDrive/Colab Notebooks/articles.csv')

In [ ]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [ ]:
articles['prod_small'] = articles['product_code'].astype('str').str[0:3]

In [ ]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,prod_small
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,108
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",110
6,111565001,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo...",111
11,112679048,112679,SWEATSHIRT OC,252,Sweater,Garment Upper body,1010001,All over pattern,7,Grey,...,G,Baby Sizes 50-98,4,Baby/Children,44,Baby Essentials & Complements,1002,Jersey Basic,Sweatshirt in soft organic cotton with a pres...,112
13,114428026,114428,Alice BANDEAU 2-p,306,Bra,Underwear,1010017,Stripe,10,White,...,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear",Two soft bandeau bras in soft jersey with side...,114


In [ ]:
articles[articles['product_code']=='624383']

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,prod_small


In [ ]:
articles = articles.drop_duplicates(['prod_small'])

In [ ]:
articles[['article_id', 'product_code', 'prod_name', 'prod_small']].to_csv('drive/MyDrive/Colab Notebooks/articles_app.csv', index=False)

In [ ]:
articles = articles[['article_id', 'product_code']]

In [ ]:
df = df.merge(articles, how='left', on='article_id')

In [ ]:
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,product_code
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,663713
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,541518
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,505221
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,685687
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,685687


In [ ]:
df.shape

(50000, 6)

In [ ]:
# df.info()

In [ ]:
# df.describe()

## Collaborative Filtering

### Creating collab filter df

In [ ]:
ohe_article = pd.get_dummies(df['product_code'])

In [ ]:
len(df.customer_id.unique())

14392

In [ ]:
article_names = list(ohe_article.columns)

In [ ]:
cf_df = pd.concat([df['customer_id'], ohe_article], axis=1)

In [ ]:
cf_df = cf_df.groupby(['customer_id'])[article_names].sum().reset_index()

In [ ]:
len(cf_df.columns)

6837

In [ ]:
cf_df.head()

,customer_id,108775,110065,111565,111586,111593,111609,116379,118458,120129,...,719610,720384,722059,723595,724281,725253,727754,728146,728162,729931
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0006d37aaf7dd84f9bbc02f6cadcb74fd72ebf370bdc5f...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Test train split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = cf_df['customer_id']
X = cf_df.drop(columns='customer_id')

In [ ]:
X.to_csv('drive/MyDrive/Colab Notebooks/app.csv', index=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [ ]:
X_train.shape

(12952, 6836)

In [ ]:
X_test.shape

(1440, 6836)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
rankings = cosine_similarity(X_test, X_train, dense_output=False)

In [ ]:
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [ ]:
recommendations = {}
for i, pred in enumerate(rankings):
    idxs = list(np.argpartition(np.array(pred), -20)[-20:])
    recs = []
    for idx in idxs:
        cid = y_train.loc[[idx]].values[0]
        rec = list(df[df['customer_id'] == cid].groupby(['article_id'])['customer_id'].count().sort_values(ascending=False).index.values.astype('int'))
        recs.extend(rec)

    recs = list(set(recs))
    
    try:
        recs = recs[:12]
    except:
        pass
    
    cur_customer = y_test.iloc[[i]].values[0]
    recommendations[cur_customer] = recs

In [ ]:
mAP_list = []
for customer in recommendations:
    top_recs = recommendations[customer]
    actual_bought = list(df[df['customer_id'] == customer].groupby(['article_id'])['customer_id'].count().sort_values(ascending=False).index.values.astype('int'))
    correct = len(set(actual_bought).intersection(set(top_recs)))
    ap = correct / len(top_recs)
    mAP_list.append(ap)

In [ ]:
np.array(mAP_list).mean()

0.09514404962321629

In [ ]:
pop_mAP_list = []
for customer in recommendations:
    pop_recs = df.groupby(['article_id'])['customer_id'].count().sort_values(ascending=False).index.values[:12]
    actual_bought = list(df[df['customer_id'] == customer].groupby(['article_id'])['customer_id'].count().sort_values(ascending=False).index.values.astype('int'))
    correct = len(set(actual_bought).intersection(set(top_recs)))
    ap = correct / len(pop_recs)
    pop_mAP_list.append(ap)

In [ ]:
np.array(pop_mAP_list).mean()

0.0013310185185185185